In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[[ 'open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)


# Split the data into training and testing sets
train_size = int(len(x_feature_tensors) * 0.67)
x_train, x_test = x_feature_tensors[:train_size], x_feature_tensors[train_size:]
y_train, y_test = y_feature_tensors[:train_size], y_feature_tensors[train_size:]



# Initialize the scaler
scaler = MinMaxScaler()

# fitting the MinMaxScaler to training data using fit_transform and then
#  test data only transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform





In [ ]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available

# Splitting the dataset into training, validation, and test sets
x_train, x_temp, y_train, y_temp = train_test_split(x_train_scaled, y_train_scaled, test_size=0.2, shuffle=False)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, shuffle=False)

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [64, 128, 256],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [4, 5, 6]
}

# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 1000, Train Loss: 9.782444249140099e-05, Validation Loss: 3.8378246244974434e-05
Iteration 2000, Train Loss: 0.000206644632271491, Validation Loss: 0.0076937745325267315
Iteration 3000, Train Loss: 0.00022622087271884084, Validation Loss: 0.013659007847309113
Iteration 4000, Train Loss: 3.227835622965358e-05, Validation Loss: 5.34421851625666e-05
Iteration 5000, Train Loss: 0.0010411026887595654, Validation Loss: 0.02351643703877926
Test Loss: 0.05946449935436249


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 3000, Train Loss: 0.0002786352124530822, Validation Loss: 0.014111456461250782
Test Loss: 0.06019523739814758
Iteration 1000, Train Loss: 4.384806379675865e-05, Validation Loss: 2.9519700547098182e-05
Iteration 2000, Train Loss: 0.0001384973293170333, Validation Loss: 0.00786834117025137
Iteration 3000, Train Loss: 0.0002506355522200465, Validation Loss: 0.014326952397823334
Iteration 4000, Train Loss: 4.724815426015994e-06, Validation Loss: 1.534068360342644e-05
Iteration 5000, Train Loss: 0.0008018998778425157, Validation Loss: 0.024684468284249306
Test Loss: 0.058925777673721313
Iteration 3000, Train Loss: 0.00021723704412579536, Validation Loss: 0.013703386299312115
Test Loss: 0.05966116860508919


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 1000, Train Loss: 0.0005549254710786045, Validation Loss: 0.00040095174335874617
Iteration 2000, Train Loss: 0.0006441919831559062, Validation Loss: 0.009928182698786259
Iteration 3000, Train Loss: 2.6933612389257178e-06, Validation Loss: 0.017625076696276665
Iteration 4000, Train Loss: 0.0002541377325542271, Validation Loss: 0.00020116388623137027
Iteration 5000, Train Loss: 0.0020724961068481207, Validation Loss: 0.050273407250642776
Test Loss: 0.06701193749904633
Iteration 1000, Train Loss: 0.0005431474419310689, Validation Loss: 0.0003913228865712881
Iteration 2000, Train Loss: 0.0006752413464710116, Validation Loss: 0.010050760582089424
Iteration 3000, Train Loss: 1.475623321312014e-06, Validation Loss: 0.01799463853240013
Iteration 4000, Train Loss: 0.00033861404517665505, Validation Loss: 0.00027820601826533675
Iteration 5000, Train Loss: 0.002602069638669491, Validation Loss: 0.052937962114810944
Test Loss: 0.06547554582357407
Iteration 1000, Train Loss: 4.27933155151

In [10]:
import torch

num_future_days= 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scalar, _ = model(last_test_window_tensor, hidden)

        # Ensure the correct shape for predicted_close_scalar
        predicted_close_scalar = predicted_close_scalar.view(1, 1, 1)

        # Create a new tensor with the desired shape [1, 3, 1]
        predicted_close_tensor = torch.zeros(1, 3, 1)
        predicted_close_tensor[:, :, 0] = predicted_close_scalar[:, 0, 0]

        print(f"Size of x_test_window: {x_test_window.size()}")
        print(f"Size of predicted_close_scalar: {predicted_close_scalar.size()}")

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, 1:], predicted_close_tensor), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform(predicted_close_tensor[0].detach().numpy())

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0]}")

        # Update x_test_window for the next iteration
        x_test_window = updated_window


Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 1: [51.569786]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 2: [51.569786]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 3: [51.569786]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 4: [51.569786]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 5: [51.569786]


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available

# Splitting the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train_scaled, y_train_scaled, test_size=0.2, shuffle=False)

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [64, 128, 256],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [4, 5, 6]
}

# Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 128,
# 'n_layers': 11, 'batch_evaluation_frequency': 7}

# Best Parameters: {'learning_rate': 0.0001, 'window_size': 10, 'hidden_dim': 256,
# 'n_layers': 10, 'batch_evaluation_frequency': 5}



# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")


    if best_params is None:
        best_params = params
        best_params['val_loss'] = val_loss.item()
    elif val_loss < best_params['val_loss']:
        best_params = params
        best_params['val_loss'] = val_loss.item()


"""
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window with the scalar value
        #x_test_window = torch.cat((x_test_window[:, 1:, :], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=1)
        x_test_window = torch.cat((x_test_window[:, :, 1:], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)


        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

print(f"Best Parameters: {best_params}")


"""


Iteration 1000, Train Loss: 7.250482303788885e-05, Validation Loss: 2.4512892196071334e-05
Iteration 2000, Train Loss: 0.00017618316633161157, Validation Loss: 0.007459477987140417
Iteration 3000, Train Loss: 0.00022018440358806401, Validation Loss: 0.013682062737643719
Iteration 4000, Train Loss: 1.9135177353746258e-05, Validation Loss: 2.3828388293622993e-05
Iteration 5000, Train Loss: 0.0009871487272903323, Validation Loss: 0.024090750142931938
Iteration 1000, Train Loss: 7.133932376746088e-05, Validation Loss: 2.2816182536189444e-05
Iteration 2000, Train Loss: 0.00016880154726095498, Validation Loss: 0.007483943365514278
Iteration 3000, Train Loss: 0.00031028524972498417, Validation Loss: 0.013572457246482372
Iteration 4000, Train Loss: 3.107694647042081e-05, Validation Loss: 2.988393862324301e-05
Iteration 5000, Train Loss: 0.0011669812956824899, Validation Loss: 0.02419096976518631
Iteration 1000, Train Loss: 8.533713116776198e-05, Validation Loss: 2.958351979032159e-05
Iteration

KeyboardInterrupt: 